In [5]:
from os import listdir
from copy import deepcopy
from Utils_BLAST import *

filedir = '/Users/harihara/Research-Activities/Data/Hot-Spring/Single_Cell_Gabe/BLAST_Hits/'

In [6]:
df_Gene_Blast_ = pd.DataFrame()
for Allele in ['A','B','C']:
    print(Allele)
    files = listdir(filedir+Allele+'_Alleles/')
    for g in files:
        if g.startswith('YSG'):
            splits = g.split('_')
            gene = splits[0]+'_'+splits[1]
            samples = listdir(filedir+Allele+'_Alleles/'+g+'/')
            for s in samples:
                if s.startswith('MS'):
                    blast_hits = listdir(filedir+Allele+'_Alleles/'+g+'/'+s+'/')
                    for b in blast_hits:
                        if b.startswith('MS'):
                            df = Load_BLAST_Hits(filedir+Allele+'_Alleles/'+g+'/'+s+'/'+b)
                            df['Sample'] = s
                            df['Allele'] = Allele
                            df['Gene'] = gene
                            df_Gene_Blast_ = df_Gene_Blast_.append(df, ignore_index = True)
            print(Allele, gene)
    

A
A YSG_1514
A YSG_0675
A YSG_0096b
A YSG_1709
A YSG_1325
A YSG_0764
A YSG_1272
A YSG_0688
A YSG_1694
A YSG_1779
A YSG_0072a
A YSG_0548
A YSG_1367
A YSG_0694b
A YSG_1299
A YSG_0966
A YSG_1319
A YSG_1924
A YSG_0108a
A YSG_0136a
A YSG_0168b
A YSG_1576
A YSG_0898
A YSG_0340
A YSG_0314
A YSG_0985
A YSG_1142
A YSG_1748
A YSG_1447b
A YSG_1487
A YSG_0699
B
B YSG_1779
B YSG_0985
B YSG_1514
B YSG_0340
B YSG_1694
B YSG_0548
B YSG_1367
B YSG_0675
B YSG_0694b
B YSG_0764
B YSG_1272
B YSG_0108a
B YSG_0096b
B YSG_1142
B YSG_0136a
B YSG_0688
B YSG_1748
B YSG_0072a
B YSG_0314
B YSG_0966
B YSG_0168b
B YSG_1299
B YSG_0898
B YSG_1325
B YSG_1924
B YSG_1487
B YSG_1319
B YSG_1576
B YSG_1709
B YSG_1447b
B YSG_0699
C
C YSG_0096b
C YSG_0072a
C YSG_1142
C YSG_0966
C YSG_1325
C YSG_1299
C YSG_1924
C YSG_0699
C YSG_0136a
C YSG_0340
C YSG_1779
C YSG_0314
C YSG_0168b
C YSG_0548
C YSG_0688
C YSG_1367
C YSG_0675
C YSG_1272
C YSG_1447b
C YSG_1514
C YSG_1709
C YSG_1748
C YSG_0985
C YSG_1694
C YSG_0898
C YSG_1319
C YSG_1

In [7]:
cov_filter = [50, 70, 75, 80, 85, 90, 95, 97, 100]
df_sample_wise = pd.DataFrame()
df_gene_wise = pd.DataFrame()

for c in cov_filter:
    df_Gene_Blast = df_Gene_Blast_[df_Gene_Blast_['qcov']>=c]
    idx = df_Gene_Blast.groupby(['Sample','Allele',
                                 'qseqid'])['bitscore'].transform(max) == df_Gene_Blast['bitscore']
    df_Gene_Blast = df_Gene_Blast.loc[idx]
    df_Gene_Blast_grp = pd.DataFrame()
    df_Gene_Blast_grp['StrainList'] = df_Gene_Blast.groupby(['Gene', 'Sample', 'qseqid'])['Allele'].apply(list)
    df_Gene_Blast_grp['Strain_String'] = df_Gene_Blast_grp['StrainList'].apply(To_String)
    df_Gene_Blast = df_Gene_Blast.set_index(['Gene','Sample','qseqid'])
    df_Gene_Blast = df_Gene_Blast.join(df_Gene_Blast_grp[['Strain_String']])
    df_Gene_Blast = df_Gene_Blast.reset_index()
    df_Gene_Blast['Cov'] = c
    #BLAST_Hits_Top_30_Alleles_1E-3_qcov_'+str(c)+'.pkl'
    df_Gene_Blast.to_pickle(filedir+'BLAST_Hits_Top_30_Alleles_qcov_'+str(c)+'.pkl')
    print(c, len(df_Gene_Blast))
    
    gene_wise_ABC, sample_wise_ABC = Pick_Minimum_SNP(df_Gene_Blast)
    gene_wise_ABC['Type'] = 'ABC'
    gene_wise_ABC['Cov'] = c
    sample_wise_ABC['Type'] = 'ABC'
    sample_wise_ABC['Cov'] = c
    df_gene_wise = df_gene_wise.append(gene_wise_ABC.reset_index(), ignore_index = True)
    df_sample_wise = df_sample_wise.append(sample_wise_ABC.reset_index(), ignore_index = True)

    gene_wise_AB, sample_wise_AB = Pick_Minimum_SNP(df_Gene_Blast[df_Gene_Blast['Allele'] != 'C'])
    gene_wise_AB['Type'] = 'AB'
    gene_wise_AB['Cov'] = c
    sample_wise_AB['Type'] = 'AB'
    sample_wise_AB['Cov'] = c
    df_gene_wise = df_gene_wise.append(gene_wise_AB.reset_index(), ignore_index = True)
    df_sample_wise = df_sample_wise.append(sample_wise_AB.reset_index(), ignore_index = True)

    print('All Reads')
    (gene_wise_all_ABC, 
     sample_wise_all_ABC) = Pick_Minimum_SNP(df_Gene_Blast[df_Gene_Blast['Strain_String'] == 'ABC'])
    gene_wise_all_ABC['Type'] = 'All_ABC'
    gene_wise_all_ABC['Cov'] = c
    sample_wise_all_ABC['Type'] = 'All_ABC'
    sample_wise_all_ABC['Cov'] = c
    df_gene_wise = df_gene_wise.append(gene_wise_all_ABC.reset_index(), ignore_index = True)
    df_sample_wise = df_sample_wise.append(sample_wise_all_ABC.reset_index(), ignore_index = True)

    (gene_wise_all_AB, 
     sample_wise_all_AB) = Pick_Minimum_SNP(df_Gene_Blast[(df_Gene_Blast['Strain_String'] == 'ABC') & 
                                                          (df_Gene_Blast['Allele'] != 'C')])
    gene_wise_all_AB['Type'] = 'All_AB'
    gene_wise_all_AB['Cov'] = c
    sample_wise_all_AB['Type'] = 'All_AB'
    sample_wise_all_AB['Cov'] = c
    df_gene_wise = df_gene_wise.append(gene_wise_all_AB.reset_index(), ignore_index = True)
    df_sample_wise = df_sample_wise.append(sample_wise_all_AB.reset_index(), ignore_index = True)

df_gene_wise.to_pickle(filedir+'BLAST_Hits_Top_30_Alleles_Gene_Wise.pkl')
df_sample_wise.to_pickle(filedir+'BLAST_Hits_Top_30_Alleles_Sample_Wise.pkl')

50 3455242
All Reads
70 3335339
All Reads
75 3298130
All Reads
80 3264169
All Reads
85 3215413
All Reads
90 3157503
All Reads
95 3022464
All Reads
97 2885078
All Reads
100 2309737
All Reads


In [ ]:
data_dir = '/Users/harihara/Mount-2/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/16S_Blast_Alignments/'
allele_types = ['Alpha','Beta','Gamma']
Alleles = ['A','B','C']
df_Alleles = pd.DataFrame()

for i in range(0, len(allele_types)):
    Allele = Alleles[i]
    samples = listdir(data_dir+allele_types[i]+'_Alleles/paf_files/')
    for s in samples:
        if s.startswith('HotsprSample'):
            sample = s.replace("HotsprSample","").replace("_FD.paf","")
            print(sample)
            df = Load_PAF(data_dir+allele_types[i]+'_Alleles/paf_files/'+s, sample, flip = True)
            df['Allele'] = Allele
            df_Alleles = df_Alleles.append(df)
df_Alleles['QCov'] = df_Alleles['AlignLength']/150.0*100
df_Alleles = df_Alleles[df_Alleles['QCov'] >= 95]
df_Alleles['Divergence'] = df_Alleles['MisMatches']/df_Alleles['QLen']*100.0
df_Alleles = df_Alleles[['Query', 'Subject','Allele','MisMatches','Divergence','Sample']]
df_Alleles = df_Alleles.rename(columns = {'Query':'qseqid', 'Subject':'Gene', 'MisMatches':'mismatch'})

In [ ]:
df_16S_Blast_grp = pd.DataFrame()
df_16S_Blast_grp['StrainList'] = df_Alleles.groupby(['Sample', 'qseqid'])['Allele'].apply(list)
df_16S_Blast_grp['Strain_String'] = df_16S_Blast_grp['StrainList'].apply(To_String)
df_Alleles = df_Alleles.set_index(['Sample','qseqid'])
df_Alleles = df_Alleles.join(df_16S_Blast_grp[['Strain_String']])
df_Alleles = df_Alleles.reset_index()

In [ ]:
df_Alleles.to_pickle(filedir+'BLAST_Hits_16S_rRNA.pkl')